In [1]:
import os
import math
import datetime
import numpy as np
import pandas as pd


import time
import os.path

## step 1 obtain OSM network

In [2]:
"Get the OSM Network"
network_type = 'auto'
import osm2gmns as og

net = og.getNetFromOSMFile('tmc.osm',network_type=(network_type), default_lanes=True, default_speed=True)

og.consolidateComplexIntersections(net)
og.outputNetToCSV(net, output_folder=os.getcwd())

osm2gmns, version 0.4.2


## step 2 Convert tmc Data into node and link

In [3]:
"Convert TMC Data into GMNS Format"
tmc = pd.read_csv('TMC_Identification.csv')

In [4]:

'''build node.csv'''
print('converting tmc data into gmns format...')


node_tmc = pd.DataFrame()
node_tmc['name'] = None
node_tmc['x_coord'] = None
node_tmc['y_coord'] = None
node_tmc['z_coord'] = None
node_tmc['node_type'] = None
node_tmc['ctrl_type'] = None
node_tmc['zone_id'] = None
node_tmc['parent_node_id'] = None
node_tmc['geometry'] = None

for i in range(0,len(tmc)-1):
    if tmc.loc[i+1,'road_order'] > tmc.loc[i,'road_order']:
        node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'start_longitude'], \
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) +")"}, ignore_index=True)
    else:
        node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc'],\
                                    'x_coord': tmc.loc[i,'start_longitude'], \
                                    'y_coord': tmc.loc[i,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) +")"}, ignore_index=True)
        node_tmc = node_tmc.append({'name': tmc.loc[i,'tmc']+'END',\
                                    'x_coord': tmc.loc[i,'end_longitude'], \
                                    'y_coord': tmc.loc[i,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i,'end_longitude'].astype(str) + " " + tmc.loc[i,'end_latitude'].astype(str) +")"}, ignore_index=True)


node_tmc = node_tmc.append({'name': tmc.loc[i+1,'tmc'],\
                                    'x_coord': tmc.loc[i+1,'start_longitude'], \
                                    'y_coord': tmc.loc[i+1,'start_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_start',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i+1,'start_longitude'].astype(str) + " " + tmc.loc[i+1,'start_latitude'].astype(str) +")"}, ignore_index=True)

node_tmc = node_tmc.append({'name': tmc.loc[i+1,'tmc']+'END',\
                                    'x_coord': tmc.loc[i+1,'end_longitude'], \
                                    'y_coord': tmc.loc[i+1,'end_latitude'],\
                                    'z_coord': None,\
                                    'node_type': 'tmc_end',\
                                    'ctrl_type': None,\
                                    'zone_id': None,\
                                    'parent_node_id': None,\
                                    'geometry': "POINT (" + tmc.loc[i+1,'end_longitude'].astype(str) + " " + tmc.loc[i+1,'end_latitude'].astype(str) +")"}, ignore_index=True)

node_tmc.index.name = 'node_id'

node_tmc.index += 100000001 #index from 0

node_tmc.to_csv('node_tmc.csv')
print('node_tmc.csv generated!')

converting tmc data into gmns format...
node_tmc.csv generated!


In [5]:
node_tmc

,name,x_coord,y_coord,z_coord,node_type,ctrl_type,zone_id,parent_node_id,geometry
node_id,,,,,,,,,
100000001,110N05711,-77.3715,39.0144,None,tmc_start,None,None,None,POINT (-77.3715 39.0144)
100000002,110-05710,-77.3709,39.0141,None,tmc_start,None,None,None,POINT (-77.3709 39.0141)
100000003,110N05710,-77.3684,39.0127,None,tmc_start,None,None,None,POINT (-77.3684 39.0127)
100000004,110-05709,-77.3605,39.0086,None,tmc_start,None,None,None,POINT (-77.3605 39.0086)
100000005,110-05708,-77.3502,39.0028,None,tmc_start,None,None,None,POINT (-77.3502 39.0028)
...,...,...,...,...,...,...,...,...,...
100000060,110+05710,-77.3500,39.0030,None,tmc_start,None,None,None,POINT (-77.35 39.003)
100000061,110P05710,-77.3609,39.0090,None,tmc_start,None,None,None,POINT (-77.3609 39.009)
100000062,110+05711,-77.3673,39.0125,None,tmc_start,None,None,None,POINT (-77.3673 39.0125)


In [6]:
'''build link_tmc.csv'''
link_tmc = pd.DataFrame()
link_tmc['name'] = None
link_tmc['corridor_id'] = None
link_tmc['corridor_link_order'] = None
link_tmc['from_node_id'] = None
link_tmc['to_node_id'] = None
link_tmc['directed'] = None
link_tmc['geometry_id'] = None
link_tmc['geometry'] = None
link_tmc['dir_flag'] = None
link_tmc['parent_link_id'] = None
link_tmc['length'] = None
link_tmc['grade'] = None
link_tmc['facility_type'] = None
link_tmc['capacity'] = None
link_tmc['free_speed'] = None
link_tmc['lanes'] = None

for i in range(0,len(tmc)):
    link_tmc = link_tmc.append({'name': tmc.loc[i,'tmc'],\
                                'corridor_id': tmc.loc[i,'road']+'_'+tmc.loc[i,'direction'],\
                                'corridor_link_order' : tmc.loc[i,'road_order'],\
                                'from_node_id': node_tmc[(node_tmc['x_coord']==tmc.loc[i,'start_longitude']) & (node_tmc['y_coord']==tmc.loc[i,'start_latitude'])].index.values[0], \
                                'to_node_id': node_tmc[(node_tmc['x_coord']==tmc.loc[i,'end_longitude']) & (node_tmc['y_coord']==tmc.loc[i,'end_latitude'])].index.values[0],\
                                'directed': 1,\
                                'geometry_id': None,\
                                'geometry': "LINESTRING (" + tmc.loc[i,'start_longitude'].astype(str) + " " + tmc.loc[i,'start_latitude'].astype(str) + "," +\
                                    tmc.loc[i,'end_longitude'].astype(str) +" "+ tmc.loc[i,'end_latitude'].astype(str) + ")",\
                                'dir_flag': 1,\
                                'parent_link_id': None,\
                                'length': tmc.loc[i,'miles'],\
                                'grade': None,\
                                'facility_type': 'interstate' if tmc.loc[i,'road'][0] == 'I'else None ,\
                                'capacity':None,\
                                'free_speed':None,\
                                'lanes': None}, ignore_index=True)
link_tmc.index.name = 'link_id'
link_tmc.index += 100000001


link_tmc.to_csv('link_tmc.csv')
print('link_tmc.csv generated!')


link_tmc.csv generated!


In [7]:
link_tmc

,name,corridor_id,corridor_link_order,from_node_id,to_node_id,directed,geometry_id,geometry,dir_flag,parent_link_id,length,grade,facility_type,capacity,free_speed,lanes
link_id,,,,,,,,,,,,,,,,
100000001,110N05711,VA-7_EASTBOUND,38.0,100000001,100000002,1,None,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",1,None,0.038620,None,None,None,None,None
100000002,110-05710,VA-7_EASTBOUND,39.0,100000002,100000003,1,None,"LINESTRING (-77.3709 39.0141,-77.3684 39.0127)",1,None,0.164895,None,None,None,None,None
100000003,110N05710,VA-7_EASTBOUND,40.0,100000003,100000004,1,None,"LINESTRING (-77.3684 39.0127,-77.3605 39.0086)",1,None,0.512158,None,None,None,None,None
100000004,110-05709,VA-7_EASTBOUND,41.0,100000004,100000005,1,None,"LINESTRING (-77.3605 39.0086,-77.3502 39.0028)",1,None,0.682589,None,None,None,None,None
100000005,110-05708,VA-7_EASTBOUND,42.0,100000005,100000006,1,None,"LINESTRING (-77.3502 39.0028,-77.3455 39.0005)",1,None,0.296731,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100000058,110+05709,VA-7_WESTBOUND,45.0,100000059,100000060,1,None,"LINESTRING (-77.346 39.001,-77.35 39.003)",1,None,0.254799,None,None,None,None,None
100000059,110+05710,VA-7_WESTBOUND,46.0,100000060,100000061,1,None,"LINESTRING (-77.35 39.003,-77.3609 39.009)",1,None,0.718593,None,None,None,None,None
100000060,110P05710,VA-7_WESTBOUND,47.0,100000061,100000062,1,None,"LINESTRING (-77.3609 39.009,-77.3673 39.0125)",1,None,0.421999,None,None,None,None,None


In [8]:
## reading by detid
reading = pd.read_csv('Reading.csv')
reading = reading.loc[0:500]


reading_dict = {}
gp = reading.groupby('_vatmc_code')
for key, form in gp:
    reading_dict[key] = {
        'measurement_tstamp':form['measurement_tstamp'].tolist(),
        'speed':form['speed'].tolist()
        }

In [9]:
'''build measurement_tmc.csv''' 

measurement_tmc = pd.DataFrame()
measurement_tmc['link_id_tmc'] = None
measurement_tmc['corridor_id'] = None
measurement_tmc['corridor_link_order'] = None
measurement_tmc['from_node_id'] = None
measurement_tmc['to_node_id'] = None
measurement_tmc['time_period'] = None
measurement_tmc['date'] = None
measurement_tmc['geometry'] = None
measurement_tmc['volume'] = None
measurement_tmc['travel_time'] = None
measurement_tmc['speed'] = None
measurement_tmc['reference_speed'] = None
measurement_tmc['density'] = None
measurement_tmc['queue'] = None
measurement_tmc['notes'] = None


k=0
measurement_tmc_dict = {}
for i in link_tmc.index:
    try:
        reading_dict_selected = reading_dict[link_tmc['name'][i]]
        for j in range(0,len(reading_dict_selected['measurement_tstamp']),3):
            measurement_tmc_dict[k] = {'link_id_tmc': i,\
                                            'corridor_id': link_tmc['corridor_id'][i],\
                                            'corridor_link_order' : link_tmc['corridor_link_order'][i],\
                                            'from_node_id': link_tmc.loc[i,'from_node_id'], \
                                            'to_node_id': link_tmc.loc[i,'to_node_id'], \
                                            'time_period': reading_dict_selected['measurement_tstamp'][j][11:13]+\
                                                reading_dict_selected['measurement_tstamp'][j][14:16]+'_'+\
                                                reading_dict_selected['measurement_tstamp'][j+3][11:13]+\
                                                reading_dict_selected['measurement_tstamp'][j+3][14:16],\
                                            'date': reading_dict_selected['measurement_tstamp'][j][:10],\
                                            'geometry': link_tmc['geometry'][i],\
                                            'volume': None,\
                                            'travel_time': None,\
                                            'speed':round(np.mean(reading_dict_selected['speed'][j:j+3])),\
                                            'reference_speed': None,\
                                            'density': None,\
                                            'queue': None,\
                                            'notes': None }
            k += 1

    except:
        measurement_tmc_dict[k] = {'link_id_tmc': i,\
                                        'corridor_id': link_tmc['corridor_id'][i],\
                                        'corridor_link_order' : link_tmc['corridor_link_order'][i],\
                                        'from_node_id': link_tmc.loc[i,'from_node_id'], \
                                        'to_node_id': link_tmc.loc[i,'to_node_id'], \
                                        'time_period': None,\
                                        'date': None,\
                                        'geometry': link_tmc['geometry'][i],\
                                        'volume': None,\
                                        'travel_time': None,\
                                        'speed': None,\
                                        'reference_speed': None,\
                                        'density': None,\
                                        'queue': None,\
                                        'notes': None }
        k += 1


    if i % 20 == 0 : print(i)
measurement_tmc = pd.DataFrame(measurement_tmc_dict).transpose()
measurement_tmc.to_csv('measurement_tmc.csv',index = False)
print('measurement_tmc.csv generated!')

100000020
100000040
100000060
measurement_tmc.csv generated!


In [10]:
measurement_tmc

,link_id_tmc,corridor_id,corridor_link_order,from_node_id,to_node_id,time_period,date,geometry,volume,travel_time,speed,reference_speed,density,queue,notes
0,100000001,VA-7_EASTBOUND,38,100000001,100000002,0000_0015,2015-04-01,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",None,None,44,None,None,None,None
1,100000001,VA-7_EASTBOUND,38,100000001,100000002,0015_0030,2015-04-01,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",None,None,44,None,None,None,None
2,100000001,VA-7_EASTBOUND,38,100000001,100000002,None,None,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)",None,None,None,None,None,None,None
3,100000002,VA-7_EASTBOUND,39,100000002,100000003,0000_0015,2015-04-01,"LINESTRING (-77.3709 39.0141,-77.3684 39.0127)",None,None,44,None,None,None,None
4,100000002,VA-7_EASTBOUND,39,100000002,100000003,0015_0030,2015-04-01,"LINESTRING (-77.3709 39.0141,-77.3684 39.0127)",None,None,44,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,100000060,VA-7_WESTBOUND,47,100000061,100000062,None,None,"LINESTRING (-77.3609 39.009,-77.3673 39.0125)",None,None,None,None,None,None,None
176,100000061,VA-7_WESTBOUND,48,100000062,100000063,0000_0015,2015-04-01,"LINESTRING (-77.3673 39.0125,-77.3707 39.0143)",None,None,46,None,None,None,None
177,100000061,VA-7_WESTBOUND,48,100000062,100000063,0015_0030,2015-04-01,"LINESTRING (-77.3673 39.0125,-77.3707 39.0143)",None,None,56,None,None,None,None
178,100000061,VA-7_WESTBOUND,48,100000062,100000063,None,None,"LINESTRING (-77.3673 39.0125,-77.3707 39.0143)",None,None,None,None,None,None,None


## step 3 mapping osm network to tmc network

In [20]:
link_gmns = pd.read_csv('link.csv', low_memory=False)

In [21]:
link_gmns

,name,link_id,osm_way_id,from_node_id,to_node_id,dir_flag,length,lanes,free_speed,capacity,link_type_name,link_type,geometry,allowed_uses,from_biway
0,NaN,0,5964128,4,5,1,146.766935,1,29,NaN,unclassified,11,"LINESTRING (-77.1878249 38.9744185, -77.187663...",auto,1
1,NaN,1,5964128,5,4,1,146.766935,1,29,NaN,unclassified,11,"LINESTRING (-77.1869767 38.9755412, -77.187013...",auto,1
2,NaN,2,5964129,6,7,1,106.872460,1,29,NaN,unclassified,11,"LINESTRING (-77.194886 38.9751513, -77.1950799...",auto,1
3,NaN,3,5964129,7,6,1,106.872460,1,29,NaN,unclassified,11,"LINESTRING (-77.1960675 38.9754343, -77.196016...",auto,1
4,NaN,4,5964129,7,8,1,85.661050,1,29,NaN,unclassified,11,"LINESTRING (-77.1960675 38.9754343, -77.196222...",auto,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30698,Sugarland Road,31152,943412232,3065,3723,1,74.086594,2,35,NaN,tertiary,5,"LINESTRING (-77.3798798 39.0078527, -77.379143...",auto,1
30699,Sugarland Road,31153,943412232,3723,3065,1,74.086594,2,35,NaN,tertiary,5,"LINESTRING (-77.37914309999999 39.0075118, -77...",auto,1
30700,Sugarland Road,31154,943412232,3723,7903,1,71.834767,2,35,NaN,tertiary,5,"LINESTRING (-77.37914309999999 39.0075118, -77...",auto,1
30701,Sugarland Road,31155,943412232,7903,3723,1,71.834767,2,35,NaN,tertiary,5,"LINESTRING (-77.37844440000001 39.0071617, -77...",auto,1


In [22]:
def LLs2Dist(lon1, lat1, lon2, lat2): #WGS84 transfer coordinate system to distance(meter) #xy #credit to xtHuang0927
    R = 6371
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0

    a = math.sin(dLat / 2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180.0) * math.cos(lat2 * math.pi / 180.0) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c * 1000
    return distance

In [23]:
matching_tmc2gmns_dict = {}
k = 0
for i in range(len(link_gmns)):
    lon_list = []
    lat_list = [] 
    link_geometry_list = link_gmns.loc[i,'geometry'][12:-1].split(", ")
    for link_geometry in link_geometry_list:
        lon_list.append(float(link_geometry.split(" ")[0]))
        lat_list.append(float(link_geometry.split(" ")[1]))
    center_lon = np.mean(lon_list)
    center_lat = np.mean(lat_list)


    distance_list = []
    for j in link_tmc.index:
        lon_tmc_list = []
        lat_tmc_list = []
        link_tmc_geometry_list = link_tmc.loc[j,'geometry'][12:-1].split(",")
        for link_tmc_geometry in link_tmc_geometry_list:
            lon_tmc_list.append(float(link_tmc_geometry.split(" ")[0]))
            lat_tmc_list.append(float(link_tmc_geometry.split(" ")[1]))
        center_tmc_lon = np.mean(lon_tmc_list)
        center_tmc_lat = np.mean(lat_tmc_list)
        
        distance_list.append(LLs2Dist(center_lon, center_lat, center_tmc_lon, center_tmc_lat))
    nearest_index = distance_list.index(min(distance_list))

    matching_tmc2gmns_dict[k] = {'name_osm':link_gmns['name'][i],\
                    'link_id_osm':link_gmns['link_id'][i],\
                    'from_node_id_osm':link_gmns['from_node_id'][i],\
                    'to_node_id_osm':link_gmns['to_node_id'][i],\
                    'geometry_osm':link_gmns['geometry'][i],\
                    'name_tmc':link_tmc.iloc[nearest_index]['name'],\
                    'link_id_tmc':int(link_tmc.iloc[[nearest_index]].index.values[0]),\
                    'from_node_id_tmc':int(link_tmc.iloc[nearest_index]['from_node_id']),\
                    'to_node_id_tmc':int(link_tmc.iloc[nearest_index]['to_node_id']),\
                    'geometry_tmc':link_tmc.iloc[nearest_index]['geometry']}
    k += 1
    if i % 5000 == 0 : print("{:.0%}".format(i/len(link_gmns))+' matching completed!')

matching_tmc2gmns = pd.DataFrame(matching_tmc2gmns_dict).transpose()
matching_tmc2gmns.to_csv('matching_tmc2gmns.csv',index = False)


0% matching completed!
16% matching completed!
33% matching completed!
49% matching completed!
65% matching completed!
81% matching completed!
98% matching completed!


In [24]:
matching_tmc2gmns

,name_osm,link_id_osm,from_node_id_osm,to_node_id_osm,geometry_osm,name_tmc,link_id_tmc,from_node_id_tmc,to_node_id_tmc,geometry_tmc
0,NaN,0,4,5,"LINESTRING (-77.1878249 38.9744185, -77.187663...",110+05700,100000046,100000047,100000048,"LINESTRING (-77.2487 38.9364,-77.2551 38.9429)"
1,NaN,1,5,4,"LINESTRING (-77.1869767 38.9755412, -77.187013...",110+05700,100000046,100000047,100000048,"LINESTRING (-77.2487 38.9364,-77.2551 38.9429)"
2,NaN,2,6,7,"LINESTRING (-77.194886 38.9751513, -77.1950799...",110+05700,100000046,100000047,100000048,"LINESTRING (-77.2487 38.9364,-77.2551 38.9429)"
3,NaN,3,7,6,"LINESTRING (-77.1960675 38.9754343, -77.196016...",110+05700,100000046,100000047,100000048,"LINESTRING (-77.2487 38.9364,-77.2551 38.9429)"
4,NaN,4,7,8,"LINESTRING (-77.1960675 38.9754343, -77.196222...",110+05700,100000046,100000047,100000048,"LINESTRING (-77.2487 38.9364,-77.2551 38.9429)"
...,...,...,...,...,...,...,...,...,...,...
30698,Sugarland Road,31152,3065,3723,"LINESTRING (-77.3798798 39.0078527, -77.379143...",110N05711,100000001,100000001,100000002,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)"
30699,Sugarland Road,31153,3723,3065,"LINESTRING (-77.37914309999999 39.0075118, -77...",110N05711,100000001,100000001,100000002,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)"
30700,Sugarland Road,31154,3723,7903,"LINESTRING (-77.37914309999999 39.0075118, -77...",110N05711,100000001,100000001,100000002,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)"
30701,Sugarland Road,31155,7903,3723,"LINESTRING (-77.37844440000001 39.0071617, -77...",110N05711,100000001,100000001,100000002,"LINESTRING (-77.3715 39.0144,-77.3709 39.0141)"


## step 4 generating measurement_osm 

In [25]:
'''build measurement_osm.csv''' 


measurement_osm = pd.DataFrame()
measurement_osm['link_id'] = None
measurement_osm['osm_way_id'] = None
measurement_osm['from_node_id'] = None
measurement_osm['to_node_id'] = None
measurement_osm['lanes'] = None
measurement_osm['length'] = None
measurement_osm['time_period'] = None
measurement_osm['date'] = None
measurement_osm['geometry'] = None
measurement_osm['volume'] = None
measurement_osm['speed'] = None


k=0
measurement_osm_dict = {}
for i in link_gmns.index:
    try:
        measurement_tmc_selected = measurement_tmc[measurement_tmc['link_id_tmc'] == matching_tmc2gmns['link_id_tmc'][i]]
        for j in measurement_tmc_selected.index:
            measurement_osm_dict[k] = {'link_id': link_gmns['link_id'][i],\
                                            'osm_way_id':link_gmns['osm_way_id'][i],\
                                            'from_node_id': link_gmns['from_node_id'][i],\
                                            'to_node_id': link_gmns['to_node_id'][i],\
                                            'lanes': link_gmns['lanes'][i], \
                                            'length': link_gmns['length'][i], \
                                            'time_period': measurement_tmc_selected['time_period'][j],\
                                            'date': measurement_tmc_selected['date'][j],\
                                            'geometry': link_gmns['geometry'][i],\
                                            'volume': None,\
                                            'speed': measurement_tmc_selected['speed'][j]}

            k += 1
    except:
        measurement_osm_dict[k] = {'link_id': link_gmns['link_id'][i],\
                                            'osm_way_id':link_gmns['osm_way_id'][i],\
                                            'from_node_id': link_gmns['from_node_id'][i],\
                                            'to_node_id': link_gmns['to_node_id'][i],\
                                            'lanes': link_gmns['lanes'][i], \
                                            'length': link_gmns['length'][i], \
                                            'time_period':None,\
                                            'date': None,\
                                            'geometry': link_gmns['geometry'][i],\
                                            'volume': None,\
                                            'speed': None}

        k += 1
    # print(i)
    if i % 5000 == 0 : print(i)
measurement_osm = pd.DataFrame(measurement_osm_dict).transpose()
measurement_osm.to_csv('measurement_osm.csv',index = False)
print('measurement_osm.csv generated!')

0
5000
10000
15000
20000
25000
30000
measurement_osm.csv generated!


In [26]:
measurement_osm

,link_id,osm_way_id,from_node_id,to_node_id,lanes,length,time_period,date,geometry,volume,speed
0,0,5964128,4,5,1,146.767,0000_0015,2015-04-01,"LINESTRING (-77.1878249 38.9744185, -77.187663...",None,40
1,0,5964128,4,5,1,146.767,0015_0030,2015-04-01,"LINESTRING (-77.1878249 38.9744185, -77.187663...",None,40
2,0,5964128,4,5,1,146.767,None,None,"LINESTRING (-77.1878249 38.9744185, -77.187663...",None,None
3,1,5964128,5,4,1,146.767,0000_0015,2015-04-01,"LINESTRING (-77.1869767 38.9755412, -77.187013...",None,40
4,1,5964128,5,4,1,146.767,0015_0030,2015-04-01,"LINESTRING (-77.1869767 38.9755412, -77.187013...",None,40
...,...,...,...,...,...,...,...,...,...,...,...
91310,31155,943412232,7903,3723,2,71.8348,0015_0030,2015-04-01,"LINESTRING (-77.37844440000001 39.0071617, -77...",None,44
91311,31155,943412232,7903,3723,2,71.8348,None,None,"LINESTRING (-77.37844440000001 39.0071617, -77...",None,None
91312,31156,943412233,12608,12607,1,27.3523,0000_0015,2015-04-01,"LINESTRING (-77.3791015 39.0131342, -77.379125...",None,44
91313,31156,943412233,12608,12607,1,27.3523,0015_0030,2015-04-01,"LINESTRING (-77.3791015 39.0131342, -77.379125...",None,44


In [27]:
print( __name__)
print(os.path.dirname(os.path.realpath('__file__')))
print(os.path.realpath('__file__'))
print(os.getcwd())

__main__
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS/__file__
/usr/local/home/ysx28/Desktop/GMNS/tmc2gmns/TMC2GMNS
